<a href="https://colab.research.google.com/github/mdzikrim/Hands-on_DL/blob/main/Chapter_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras import layers

# Build simple model
model = tf.keras.Sequential([
    layers.Input(shape=(28, 28)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Dummy training
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255.0
model.fit(x_train, y_train, epochs=1, batch_size=32)

# Export SavedModel
model.export("mnist_model")

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8609 - loss: 0.4988
Saved artifact at 'mnist_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  136815924913616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136815924914000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136812916247056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136812916252240: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [4]:
# Jalankan ini di terminal lokal (bukan Colab)
!docker pull tensorflow/serving
!docker run -p 8501:8501 --name=tf_model \
  --mount type=bind,source=$(pwd)/mnist_model,target=/models/mnist_model \
  -e MODEL_NAME=mnist_model -t tensorflow/serving

/bin/bash: line 1: docker: command not found
/bin/bash: line 1: docker: command not found


In [5]:
import json
import requests
import numpy as np

# Pick sample
sample = x_train[0].tolist()
data = json.dumps({"signature_name": "serving_default", "instances": [sample]})

# POST request
url = "http://localhost:8501/v1/models/mnist_model:predict"
headers = {"content-type": "application/json"}
response = requests.post(url, data=data, headers=headers)

# Prediction
predictions = response.json()['predictions']
print("Predicted class:", np.argmax(predictions[0]))


ConnectionError: HTTPConnectionPool(host='localhost', port=8501): Max retries exceeded with url: /v1/models/mnist_model:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c6e3ab2c210>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [6]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation='relu'),
        layers.Dense(10)
    ])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer='adam', metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=5, batch_size=256)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7756 - loss: 0.7917
Epoch 2/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9375 - loss: 0.2253
Epoch 3/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9532 - loss: 0.1665
Epoch 4/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9632 - loss: 0.1299
Epoch 5/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9707 - loss: 0.1030


In [7]:
strategy = tf.distribute.experimental.CentralStorageStrategy()
with strategy.scope():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation='relu'),
        layers.Dense(10)
    ])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer='adam', metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=5, batch_size=256)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7956 - loss: 0.7509
Epoch 2/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9353 - loss: 0.2291
Epoch 3/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9530 - loss: 0.1682
Epoch 4/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9659 - loss: 0.1232
Epoch 5/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9705 - loss: 0.1034
